# **INITIAL STEPS**

In [1]:
!pip install transformers

In [2]:
import torch

from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# **PERSONAL TESTING**

In [6]:
# question = "How many parameters does BERT-large have?"
# answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [7]:
# # Apply the tokenizer to the input text, treating them as a text-pair.
# input_ids = tokenizer.encode(question, answer_text, max_length=5000, truncation = True)

# print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 70 tokens.


In [8]:
# # Search the input_ids for the first instance of the `[SEP]` token.
# sep_index = input_ids.index(tokenizer.sep_token_id)

# # The number of segment A tokens includes the [SEP] token istelf.
# num_seg_a = sep_index + 1

# # The remainder are segment B.
# num_seg_b = len(input_ids) - num_seg_a

# # Construct the list of 0s and 1s.
# segment_ids = [0]*num_seg_a + [1]*num_seg_b

# # There should be a segment_id for every input token.
# assert len(segment_ids) == len(input_ids)

In [9]:
# # Run our example through the model.
# start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
#                                  token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text


In [10]:
# tokens = tokenizer.convert_ids_to_tokens(input_ids)

In [11]:
# # Find the tokens with the highest `start` and `end` scores.
# answer_start = torch.argmax(start_scores)
# answer_end = torch.argmax(end_scores)

# # Start with the first token.
# answer = tokens[answer_start]

# # Select the remaining answer tokens and join them with whitespace.
# for i in range(answer_start + 1, answer_end + 1):
    
#     # If it's a subword token, then recombine it with the previous token.
#     if tokens[i][0:2] == '##':
#         answer += tokens[i][2:]
    
#     # Otherwise, add a space then the token.
#     else:
#         answer += ' ' + tokens[i]

# print('Answer: "' + answer + '"')

Answer: "340m"


# **QA FUNCTIONS BELOW**


In [53]:
def check_split (input_ids):
    ''' 
    Splits data if there are more than 512 tokens, which is BERT's 
    limit context input
    '''
    print("length of input_ids: "+str(len(input_ids)))
    # Keep track of answer splits
    answer_splits = []
    # Total number of splits due to the number of tokens and BERT limit of 512
    num_of_splits = 0
    # Number of tokens
    len_input = len(input_ids)

    # Determine number of splits required
    if (len_input > 512):
        if len_input % 512 == 0:
            num_of_splits = len_input / 512
        else:
            num_of_splits = int(len_input /  512) + 1

    print("number of splits: "+str(num_of_splits))

    temp_ids = input_ids
     # Compute answer for all splits
    # for i in range(0, num_of_splits+1):
    for i in range(1):
        ind = (i+1)*512
        text_split_i = temp_ids[:ind]
        print("ind: "+str(ind))
        print("Computing answer for split i: "+ str(text_split_i))
        answer_splits.append(split_answer(text_split_i))
        temp_ids = input_ids[ind:]
    print("testing to see if rest still works")
    # print(split_answer(input_ids))
    
    return answer_splits


def answer_question(question, answer_text):
    '''
    Tokenizes the question and answer text
    '''

    input_ids = tokenizer.encode(question, answer_text)
    check_split (input_ids)

                                                                   

def split_answer (input_ids):
    '''
    Determines answer start and ending indexes for split_input
    '''
    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    answer = full_sentence (tokens, answer_start, answer_end)
    

    print('Answer: "' + answer + '"')

In [54]:
def start_of_sentence (tokens, answer_start, answer_end):

    # Start with the first token and/or first letter in sentence
    if tokens[answer_start-1] == '.' or tokens[answer_start] == None:
        answer = tokens[answer_start]
    
    # Otherwise keep searching for the beginning of the sentence
    else: 
        while tokens[answer_start].isalnum() == False:                                    # ADD OR StATEMENT TOO??
            print(tokens[answer_start])
            answer_start-=1
    
    # Find the last letter in the sentence
    if tokens[answer_end].isalnum() == False:                                              # ADD OR StATEMENT TOO??
        while (tokens[answer_end] != '.' or answer_end+1 >= len(tokens)) and (len(tokens) > answer_end+1):
            answer_end+=1

    return tokens[answer_start], answer_start, answer_end

def update_tokens(answer, tokens, answer_start, answer_end):

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # If there is a punctuation mark, then combine without space
        elif tokens[i].isalnum() == False:
            answer += tokens[i]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    
    return answer

def update_punctuation (answer):
    
    # Set beginning of sentence to a letter
    while answer[0].isalnum() == False:
        answer = answer[1:]

    # Set end of sentence to a letter
    len_answer = len(answer)
    while answer[len_answer-1].isalnum() == False:
        answer = answer[:len_answer-1]
        len_answer-=1
    
    return answer

def full_sentence (tokens, answer_start, answer_end):

    # Update indices to include full sentence
    new_res = start_of_sentence(tokens, answer_start, answer_end)
    answer = new_res[0]
    answer_start = new_res[1]
    answer_end = new_res[2]

    # Update answer to include correct spacing with punctuation and combine ## tokens
    answer = update_tokens(answer, tokens, answer_start, answer_end)

    # Remove extraneous punctuation from start and end
    answer = update_punctuation(answer)

    # Capitalize the first letter and add period punctuation to end
    return answer[0].upper()+answer[1:]+'.'


In [55]:
bert_abstract = "Purchased Content. When you purchase an item of content, your content will be stored in a digital locker and you may view it an unlimited number of times for during your Locker Period. The “Locker Period” will be for at least 5 years from the date of your purchase (subject to the restrictions described in the YouTube Paid Service Terms of Service). Each item of purchased content may have a different Locker Period and you agree to the Locker Period before you order it. Pausing, stopping, or rewinding purchased content will not extend the Locker Period.  As noted in the YouTube Paid Service Terms of Service, if an item of purchased content becomes unavailable during the five year period from the purchase date, you may request a refund. For purchased content: you may view one stream of each item at a time, you may view up to 3 streams of different items at a time, you may authorize up to 5 devices for offline playback of Locker Video Content at a time and to authorize additional devices, you must deauthorize one of those 5 devices, you may only authorize the same device three times in any 12 month period and de-authorize the same device twice in any 12 month period, you may only deauthorize a total of 2 devices for offline playback every 90 days, and you may only authorize 3 Google accounts on the same device. Stream and offline playback limitations for purchased content apply regardless of which Google product (e.g., Google Play Movies & TV or YouTube) you access the content from."
question = "Can I get a refund?"

answer_question(question, bert_abstract)


length of input_ids: 323
number of splits: 0
ind: 512
Computing answer for split i: [101, 2064, 1045, 2131, 1037, 25416, 8630, 1029, 102, 4156, 4180, 1012, 2043, 2017, 5309, 2019, 8875, 1997, 4180, 1010, 2115, 4180, 2097, 2022, 8250, 1999, 1037, 3617, 12625, 1998, 2017, 2089, 3193, 2009, 2019, 14668, 2193, 1997, 2335, 2005, 2076, 2115, 12625, 2558, 1012, 1996, 1523, 12625, 2558, 1524, 2097, 2022, 2005, 2012, 2560, 1019, 2086, 2013, 1996, 3058, 1997, 2115, 5309, 1006, 3395, 2000, 1996, 9259, 2649, 1999, 1996, 7858, 3825, 2326, 3408, 1997, 2326, 1007, 1012, 2169, 8875, 1997, 4156, 4180, 2089, 2031, 1037, 2367, 12625, 2558, 1998, 2017, 5993, 2000, 1996, 12625, 2558, 2077, 2017, 2344, 2009, 1012, 20490, 1010, 7458, 1010, 2030, 2128, 11101, 2075, 4156, 4180, 2097, 2025, 7949, 1996, 12625, 2558, 1012, 2004, 3264, 1999, 1996, 7858, 3825, 2326, 3408, 1997, 2326, 1010, 2065, 2019, 8875, 1997, 4156, 4180, 4150, 20165, 2076, 1996, 2274, 2095, 2558, 2013, 1996, 5309, 3058, 1010, 2017, 2089, 5227, 

# **WEB SCRAPING**

In [41]:
# from bs4 import BeautifulSoup
# from bs4.element import Comment
# import urllib.request


# def tag_visible(element):
#     if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
#         return False
#     if isinstance(element, Comment):
#         return False
#     return True


# def text_from_html(body):
#     soup = BeautifulSoup(body, 'html.parser')
#     texts = soup.findAll(text=True)
#     visible_texts = filter(tag_visible, texts)  
#     return u" ".join(t.strip() for t in visible_texts)
# # youtube: https://www.youtube.com/t/usage_paycontent
# # tinder: https://policies.tinder.com/terms/us/en
# html = urllib.request.urlopen('https://policies.tinder.com/terms/us/en').read()
# print(text_from_html(html))

Language English Español(MX) Company Jobs Contact Security Community Blog Tech Blog Press Support Partners Hey! Vina Spotify Legal Privacy Terms Cookie Policy Safety and Policy Center Safety Tips Community Guidelines Intellectual Property Terms of Use  Last revised on April 28, 2020  Welcome to Tinder, operated by Match Group, LLC (“us,” “we,” the “Company” or “Tinder”).  Notice to California subscribers: You may cancel your subscription, without penalty or obligation, at any time prior to midnight of the third business day following the date you subscribed. If you subscribed using your Apple ID, refunds are handled by Apple, not Tinder. If you wish to request a refund, please visit https://getsupport.apple.com . If you subscribed using your Google Play Store account or through Tinder Online: Contact customer support  1. Acceptance of Terms of Use Agreement.  By creating a Tinder account or by using any Tinder service, whether through a mobile device, mobile application or computer (co

In [56]:
bert_abstract = text_from_html(html)
question = "Can I get a refund?"

answer_question(question, bert_abstract)

length of input_ids: 7944
number of splits: 16
ind: 512
Computing answer for split i: [101, 2064, 1045, 2131, 1037, 25416, 8630, 1029, 102, 2653, 2394, 9686, 9739, 4747, 1006, 25630, 1007, 2194, 5841, 3967, 3036, 2451, 9927, 6627, 9927, 2811, 2490, 5826, 4931, 999, 19354, 2050, 3962, 8757, 3423, 9394, 3408, 17387, 3343, 3808, 1998, 3343, 2415, 3808, 10247, 2451, 11594, 7789, 3200, 3408, 1997, 2224, 2197, 8001, 2006, 2258, 2654, 1010, 12609, 6160, 2000, 9543, 4063, 1010, 3498, 2011, 2674, 2177, 1010, 11775, 1006, 1523, 2149, 1010, 1524, 1523, 2057, 1010, 1524, 1996, 1523, 2194, 1524, 2030, 1523, 9543, 4063, 1524, 1007, 1012, 5060, 2000, 2662, 17073, 1024, 2017, 2089, 17542, 2115, 15002, 1010, 2302, 6531, 2030, 14987, 1010, 2012, 2151, 2051, 3188, 2000, 7090, 1997, 1996, 2353, 2449, 2154, 2206, 1996, 3058, 2017, 4942, 29234, 2094, 1012, 2065, 2017, 4942, 29234, 2094, 2478, 2115, 6207, 8909, 1010, 25416, 26698, 2024, 8971, 2011, 6207, 1010, 2025, 9543, 4063, 1012, 2065, 2017, 4299, 2000, 